In [1]:
cd drive/MyDrive/Dacon/LDI/Model

/content/drive/MyDrive/Dacon/LDI/Model


In [2]:
import pickle
import pandas as pd
import numpy as np
with open('../Preprocessed_data/list.txt', 'rb') as f:
    problem_df = pickle.load(f)
with open('../Preprocessed_data/noProblemList.txt', 'rb') as f:
    no_problem_df = pickle.load(f)

In [3]:
problem_df

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,19224.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,23664.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.111111,-0.111111,-0.111111,0.0,0.0,0.888889,-0.111111,0.0,0.0,0.0,3.222222,-0.111111,0.0
2,23664.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.285714,-0.285714,-0.285714,0.0,0.0,0.428571,-0.285714,0.0,0.0,0.0,2.571429,-0.285714,0.0
3,15166.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.910508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.250000,-0.250000,-0.250000,0.0,0.0,0.500000,-0.250000,0.0,0.0,0.0,3.000000,-0.250000,0.0
4,12590.0,1.0,1.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6234,20167.0,0.0,3.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6235,20167.0,0.0,4.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6236,16270.0,2.0,5.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6237,19114.0,3.0,9.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


In [4]:
no_problem_df

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10002.0,3.0,9.0,0.0,0.0,0.0,0.0,36.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,-0.045455,0.000000,0.0,0.0,1.363636,2.045455,4.363636,0.0,0.090909,4.318182,-0.045455,0.0
1,10003.0,2.0,5.0,0.0,0.0,0.0,0.0,39.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2307.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,10003.0,2.0,6.0,0.0,0.0,0.0,0.0,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1892.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,10007.0,0.0,3.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,10007.0,0.0,4.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13763,24992.0,0.0,4.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
13764,24993.0,3.0,9.0,0.0,0.0,0.0,0.0,36.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,-0.333333,-0.333333,0.0,0.0,1.000000,-0.333333,0.000000,0.0,0.000000,4.000000,-0.333333,0.0
13765,24994.0,3.0,9.0,0.0,0.0,0.0,0.0,36.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
13766,24995.0,2.0,6.0,0.0,0.0,0.0,0.0,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,-0.333333,-0.333333,0.0,0.0,0.500000,-0.333333,0.000000,0.0,0.000000,2.500000,-0.333333,0.0


In [5]:
df_x = pd.concat([problem_df,no_problem_df])
df_x

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,19224.0,0.0,3.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,23664.0,0.0,3.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.111111,-0.111111,-0.111111,0.0,0.0,0.888889,-0.111111,0.0,0.0,0.0,3.222222,-0.111111,0.0
2,23664.0,0.0,4.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.285714,-0.285714,-0.285714,0.0,0.0,0.428571,-0.285714,0.0,0.0,0.0,2.571429,-0.285714,0.0
3,15166.0,0.0,3.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.910508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.250000,-0.250000,-0.250000,0.0,0.0,0.500000,-0.250000,0.0,0.0,0.0,3.000000,-0.250000,0.0
4,12590.0,1.0,1.0,0.0,0.0,0.0,0.0,39.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13763,24992.0,0.0,4.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
13764,24993.0,3.0,9.0,0.0,0.0,0.0,0.0,36.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,-0.333333,-0.333333,0.0,0.0,1.000000,-0.333333,0.0,0.0,0.0,4.000000,-0.333333,0.0
13765,24994.0,3.0,9.0,0.0,0.0,0.0,0.0,36.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
13766,24995.0,2.0,6.0,0.0,0.0,0.0,0.0,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,-0.333333,-0.333333,0.0,0.0,0.500000,-0.333333,0.0,0.0,0.0,2.500000,-0.333333,0.0


In [6]:
y_problem = np.zeros((13768))
y_no_problem = np.ones((6239))
df_y = np.concatenate((y_problem,y_no_problem))
df_y

array([0., 0., 0., ..., 1., 1., 1.])

In [59]:
from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(df_x,df_y, test_size = 0.3, random_state = 11)
X_train, X_val,y_train, y_val = train_test_split(X_t,y_t,test_size=0.2, random_state = 10)
X_train

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
2615,16293.0,1.0,1.0,3.0,0.0,0.0,0.0,39.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.994819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.062500,-0.062500,-0.062500,0.0,0.0,49.312500,0.000000,0.1875,0.0,0.0,194.4375,-0.062500,0.0
8814,19526.0,2.0,6.0,0.0,0.0,0.0,0.0,7.080537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.585366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.0,0.0000,0.000000,0.0
2154,18159.0,2.0,6.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.0,0.0000,0.000000,0.0
4540,19750.0,1.0,1.0,0.0,0.0,0.0,0.0,39.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2059.0,0.0,0.0,0.0,0.0,-0.250000,-0.250000,-0.250000,0.0,0.0,0.500000,-0.250000,0.0000,0.0,0.0,3.0000,-0.250000,0.0
5349,15808.0,0.0,3.0,0.0,0.0,0.0,0.0,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,-0.333333,-0.333333,0.0,0.0,1.333333,-0.333333,0.0000,0.0,0.0,5.0000,-0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7606,18242.0,2.0,5.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.0,0.0000,0.000000,0.0
659,11758.0,0.0,3.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.885501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.0,0.0000,0.000000,0.0
4855,18937.0,2.0,6.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.0,0.0000,0.000000,0.0
2610,19747.0,0.0,4.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.0,0.0000,0.000000,0.0


In [8]:
y_train

array([0., 0., 1., ..., 0., 0., 0.])

In [11]:
!pip install -q xgboost

In [61]:
from xgboost import plot_importance
from xgboost import XGBClassifier

eval_set=[(X_val,y_val)]
xgb = XGBClassifier(n_estimators = 500, learning_rate = 0.05, max_depth = 10, objective= 'binary:logistic')
xgb.fit(X_train,y_train,eval_set = eval_set, eval_metric='auc',early_stopping_rounds = 20,verbose = True)
xgb_pred = xgb.predict(X_test)
xgb_probability = xgb.predict_proba(X_test)

[0]	validation_0-auc:0.92247
[1]	validation_0-auc:0.92334


/usr/local/lib/python3.6/dist-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[2]	validation_0-auc:0.92318
[3]	validation_0-auc:0.92406
[4]	validation_0-auc:0.92425
[5]	validation_0-auc:0.92453
[6]	validation_0-auc:0.92514
[7]	validation_0-auc:0.92547
[8]	validation_0-auc:0.92559
[9]	validation_0-auc:0.92549
[10]	validation_0-auc:0.92555
[11]	validation_0-auc:0.92561
[12]	validation_0-auc:0.92554
[13]	validation_0-auc:0.92574
[14]	validation_0-auc:0.92542
[15]	validation_0-auc:0.92557
[16]	validation_0-auc:0.92556
[17]	validation_0-auc:0.92574
[18]	validation_0-auc:0.92553
[19]	validation_0-auc:0.92566
[20]	validation_0-auc:0.92576
[21]	validation_0-auc:0.92594
[22]	validation_0-auc:0.92615
[23]	validation_0-auc:0.92615
[24]	validation_0-auc:0.92624
[25]	validation_0-auc:0.92625
[26]	validation_0-auc:0.92631
[27]	validation_0-auc:0.92592
[28]	validation_0-auc:0.92593
[29]	validation_0-auc:0.92605
[30]	validation_0-auc:0.92604
[31]	validation_0-auc:0.92627
[32]	validation_0-auc:0.92633
[33]	validation_0-auc:0.92615
[34]	validation_0-auc:0.92618
[35]	validation_0-

In [63]:
xgb_prob = xgb_probability[:,1]
xgb_prob

array([0.7971881 , 0.02860597, 0.02860597, ..., 0.02860597, 0.02860597,
       0.02860597], dtype=float32)

In [64]:

from sklearn.metrics import roc_auc_score


roc_auc_score(y_test,xgb_pred)
roc_auc_score(y_test,xgb_prob)


0.9367552041391347

In [65]:
with open('../Preprocessed_data/x_test.txt', 'rb') as f:
    testdata = pickle.load(f)

In [66]:
test_pred = xgb.predict_proba(testdata)[:,1]

In [67]:
test_pred

array([0.5483622 , 0.56719995, 0.32288998, ..., 0.08003656, 0.7515149 ,
       0.4726984 ], dtype=float32)

In [68]:
seq_user_id = testdata['user_id']
seq_user_id
res_dict = {}
for i,user_id in enumerate(seq_user_id):
    user_id = int(user_id)
    if user_id not in res_dict.keys():
        res_dict[user_id] = test_pred[i]
    else:
        if res_dict[user_id] < test_pred[i]:
            res_dict[user_id] = test_pred[i]
res_dict

{30000: 0.56719995,
 30001: 0.58346,
 30002: 0.756668,
 30003: 0.6667749,
 30004: 0.5938541,
 30005: 0.7138451,
 30006: 0.7286977,
 30007: 0.82066303,
 30008: 0.73192227,
 30010: 0.8702971,
 30011: 0.7286977,
 30012: 0.69742286,
 30013: 0.7592681,
 30014: 0.7004603,
 30015: 0.78411096,
 30016: 0.8702971,
 30017: 0.49661866,
 30018: 0.41808146,
 30019: 0.35366577,
 30020: 0.6297508,
 30021: 0.78411096,
 30022: 0.8462563,
 30023: 0.71512014,
 30024: 0.8963343,
 30025: 0.7845933,
 30026: 0.7732765,
 30027: 0.71623385,
 30028: 0.63303393,
 30029: 0.600475,
 30030: 0.42973164,
 30031: 0.7004603,
 30032: 0.7815081,
 30033: 0.792435,
 30034: 0.7867022,
 30035: 0.72221047,
 30036: 0.7186766,
 30037: 0.81172615,
 30038: 0.7408669,
 30039: 0.7592681,
 30040: 0.5481269,
 30041: 0.68029773,
 30042: 0.7286977,
 30043: 0.66827685,
 30044: 0.72613984,
 30045: 0.5582498,
 30046: 0.5546218,
 30047: 0.46653837,
 30048: 0.5798051,
 30049: 0.86243266,
 30050: 0.6787698,
 30051: 0.7862697,
 30052: 0.516739

In [70]:
import csv

with open('result.csv','w') as csvfile:
    fieldnames = ['user_id','problem']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(30000,44999):
        if i not in res_dict.keys():
            writer.writerow({'user_id':i,'problem':0})
        else:
            writer.writerow({'user_id':i,'problem':res_dict[i]})

In [71]:
fieldnames = ['user_id','problem']

In [72]:
len(res_dict)

14401